In [1]:
from huggingface_hub import notebook_login
from datasets import Dataset
from datasets import DatasetDict
from datasets import Features
from datasets import ClassLabel
from datasets import Sequence
import torch
from transformers import AutoTokenizer, AutoModel, BertConfig
from transformers import AutoTokenizer, TFEsmForTokenClassification
from transformers import TFAutoModelForTokenClassification
from transformers import DataCollatorForTokenClassification
import evaluate
from transformers import create_optimizer
import numpy as np
import tensorflow as tf
from transformers.keras_callbacks import KerasMetricCallback
from transformers import pipeline
import pandas as pd
# import AutoModelForCausalLM

2024-10-30 17:52:52.993377: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-30 17:52:53.004334: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-30 17:52:53.007743: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-30 17:52:53.016493: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-30 17:52:55.066918: W tensorflow/compiler/tf2

In [2]:
notebook_login()

In [3]:
# from transformers import BertModel
# device = "cuda:0" if torch.cuda.is_available() else "cpu"
# sentence  = 'Hello World!'
# tokenizer = AutoTokenizer.from_pretrained('bert-large-uncased')
# # model     = BertModel.from_pretrained('bert-large-uncased')

# # inputs    = tokenizer(sentence, return_tensors="pt").to(device)
# # model     = model.to(device)
# # outputs   = model(**inputs)


In [1]:
print("done")

done


In [5]:
id2label = {
    0:"a",
    1:"b"
}
label2id ={
    "a" : 0,
    "b" : 1
}

labels = [0,1]
label_list = ["0","1"]
SAMPLE_TEXT = "ATATATATATATATATATATATATATATAT"
NAMES = ["EXAMPLE","TARGETSCAN"]
EXAMPLE_DATASET = 0
TARGETSCAN_DATASET = 1
CURRENT_DATASET = EXAMPLE_DATASET

MODEL_NAMES = ["zhihan1996/DNABERT-2-117M","InstaDeepAI/nucleotide-transformer-500m-human-ref"]
DNABERT_MODEL = 0
SEGMENT_MODEL = 1
CURRENT_MODEL = DNABERT_MODEL
MODEL_NAME = MODEL_NAMES[SEGMENT_MODEL]
NUM_OF_EPOCH = 3
SMALL_MODEL = False
SAVE_MODEL_NAME = f"model_{MODEL_NAMES}_dataset_{NAMES[CURRENT_DATASET]}_num_of_epoch_{NUM_OF_EPOCH}"


In [6]:
if SMALL_MODEL:
    SAVE_MODEL_NAME += "_small"
SAVE_MODEL_NAME

"model_['zhihan1996/DNABERT-2-117M', 'InstaDeepAI/nucleotide-transformer-500m-human-ref']_dataset_EXAMPLE_num_of_epoch_3"

# Dataset Creation

## Example Dataset

In [7]:
SIZE_OF_DATASET = 300
import random


# example dataset
MIN_NUM_OF_TOKENS = 30
MAX_NUM_OF_TOKENS = 400
MIN_SIZE_OF_TOKEN = 3
MAX_SIZE_OF_TOKEN = 10
REAL_TOKEN = "GCGCGCGCGCGCGCGC"
FAKE_TOKEN = "ATATATATATATATAT"

# Generate a random integer between 1 and 10 (inclusive)

def create_dataset(number_of_elements : int, curr_index : int):
    
    tokens = []
    ner_tags = []
    ids = []
    for i in range(number_of_elements):
        num_of_tokens = random.randint(MIN_NUM_OF_TOKENS, MAX_NUM_OF_TOKENS)
        length_of_token = random.randint(MIN_SIZE_OF_TOKEN, MAX_SIZE_OF_TOKEN)
        tokens.append([REAL_TOKEN[:length_of_token],FAKE_TOKEN[:length_of_token]])
        ner_tags.append([1,0])
        ids.append(curr_index)
        curr_index+=1
    ds = Dataset.from_dict(
    {
        "tokens": tokens, 
        "ner_tags": ner_tags,
        "id":ids,
    })
    return ds
    # pass

def create_dataset_dict(number_of_elements : int):
    train_ds = create_dataset(number_of_elements,0)
    validation_ds = create_dataset(number_of_elements,number_of_elements)
    test_ds = create_dataset(number_of_elements,number_of_elements*2)
    
    ds = DatasetDict({"train": train_ds, "validation" : validation_ds, "test":test_ds})
    return ds

    

## Targetscan dataset

In [8]:
TARGETSCAN_DATASET_PATH = "/sci/nosnap/michall/roeizucker/token_classification/targetscan_RNA_combined_nrows_90000_hg38.csv"
def create_targetscan_dataset(df,start,end):
    tokens = df["result"][start:end]
    ner_tags = df["stags"][start:end]
    ids = df["Unnamed: 0"][start:end]
    ds = Dataset.from_dict(
    {
        "tokens": tokens, 
        "ner_tags": ner_tags,
        "id":ids,
    })
    return ds

def create_targetscan_dataset_dict(df):
    train_ds = create_targetscan_dataset(df,2,len(df) // 2 + 10)
    validation_ds = create_targetscan_dataset(df,len(df) // 2 + 11,(len(df) // 4) * 3)
    test_ds = create_targetscan_dataset(df,(len(df) // 4) * 3,(len(df) // 4) * 4 - 2)
    
    ds = DatasetDict({"train": train_ds, "validation" : validation_ds, "test":test_ds})
    return ds



In [9]:
!ls /sci/nosnap/michall/roeizucker/token_classification/

targetscan_RNA_combined_nrows_90000.csv
targetscan_RNA_combined_nrows_90000_hg38.csv


# Code

In [10]:
if CURRENT_DATASET == EXAMPLE_DATASET:
    ds = create_dataset_dict(SIZE_OF_DATASET)
elif CURRENT_DATASET == TARGETSCAN_DATASET:
    df = pd.read_csv(TARGETSCAN_DATASET_PATH)
    
    SAMPLE_TEXT = df["result"][100].replace("'","").replace(" ","").replace("[","").replace("]","")
    df["result"] = df["result"].str.replace("'","")
    df["result"] = df["result"].apply(lambda x:x.strip('][').split(', '))
    df["stags"] = df["stags"].apply(lambda x:x.strip('][').split(', '))
    df['stags'] = df['stags'].apply(lambda x: list(map(int, x)))
    df = df[df["result"].map(len) < 20]
    ds = create_targetscan_dataset_dict(df)
else:
    raise Exception("no such dataset") 

In [11]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

device = "cuda:0" if torch.cuda.is_available() else "cpu"

# config = BertConfig.from_pretrained("zhihan1996/DNABERT-2-117M",hidden_size=400, num_hidden_layers=4, num_attention_heads=4,intermediate_size=35,max_position_embeddings=10000)

# if SMALL_MODEL:
#     config = BertConfig.from_pretrained(MODEL_NAME,hidden_size=800, num_hidden_layers=8, num_attention_heads=8,intermediate_size=70,max_position_embeddings=10000)
# else:
#     config = BertConfig.from_pretrained(MODEL_NAME,max_position_embeddings=10000)
# # model = AutoModel.from_config(config)

# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# model = TFAutoModelForTokenClassification.from_config(config)
example = ds["train"][0]
print(example)
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokenized_ds = ds.map(tokenize_and_align_labels, batched=True)

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)


seqeval = evaluate.load("seqeval")


# example = ds["train"][0]
# label_list = ds["train"].features[f"ner_tags"].feature.names

batch_size = 16
num_train_epochs = NUM_OF_EPOCH
num_train_steps = (len(tokenized_ds["train"]) // batch_size) * num_train_epochs
optimizer, lr_schedule = create_optimizer(
    init_lr=2e-5,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
    num_warmup_steps=0,
)
tf_train_set = model.prepare_tf_dataset(
    tokenized_ds["train"],
    shuffle=True,
    batch_size=2,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_ds["validation"],
    shuffle=False,
    batch_size=2,
    collate_fn=data_collator,
)
model.compile(optimizer=optimizer)  # No loss argument!
metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)
callbacks = [metric_callback]
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=NUM_OF_EPOCH, callbacks=callbacks)


# change evaluation to something generic
classifier = pipeline("ner", model=model,tokenizer=tokenizer)
print("text",SAMPLE_TEXT,"size",SIZE_OF_DATASET)
for val in classifier(SAMPLE_TEXT):
    print(val)

{'tokens': ['GCGCGCGCG', 'ATATATATA'], 'ner_tags': [1, 0], 'id': 0}


NameError: name 'tokenizer' is not defined

In [ ]:
# dir(model.to)
print(tf.config.list_physical_devices())


In [ ]:
print("A")

In [ ]:
# classifier = pipeline("ner", model=model,tokenizer=tokenizer)
# print("text",SAMPLE_TEXT,"size",SIZE_OF_DATASET)
for val in classifier(''.join(tokenized_ds["train"][2]["tokens"])):
    print(val)

In [ ]:
tokenized_ds["train"][2]

In [ ]:
TESTED_VAL = 101
tokens = []
result = classifier(''.join(tokenized_ds["train"][TESTED_VAL]["tokens"]))
for i in range(len(tokenized_ds["train"][TESTED_VAL]["tokens"])):
    tokens.extend([str(tokenized_ds["train"][TESTED_VAL]["ner_tags"][i])] * len(tokenized_ds["train"][TESTED_VAL]["tokens"][i]))
print("code\t\t",''.join(tokenized_ds["train"][TESTED_VAL]["tokens"]))
print("origin\t\t",''.join(tokens))

pred = []
for val in result:
    pred.extend([val["entity"][-1]]*len(val["word"]))

print("predicted\t",''.join(pred))



print("predicted")
for val in result:
    if val["entity"] == 'LABEL_1':
        print(val)

print("source")
count = 0
for i in range(len(tokenized_ds["train"][TESTED_VAL]["tokens"])):
    if tokenized_ds["train"][TESTED_VAL]["ner_tags"][i] == 1:
        print(count,"-",end="")
    count += len(tokenized_ds["train"][TESTED_VAL]["tokens"][i])
    if tokenized_ds["train"][TESTED_VAL]["ner_tags"][i] == 1:
        print(count)

In [ ]:
# tokenized_ds["train"][0]
''.join(tokenized_ds["train"][2]["tokens"])

In [ ]:
tokenized_ds["train"][2]

In [ ]:
classifier.save_pretrained(SAVE_MODEL_NAME)

In [ ]:
# model = AutoModelForCausalLM.from_pretrained("modeldir")
# print("done")

In [ ]:
pipe = pipeline("ner", model = SAVE_MODEL_NAME,device=1)

In [ ]:
for val in pipe(''.join(tokenized_ds["train"][2]["tokens"])):
    print(val)